In [14]:
from langchain_experimental.agents.agent_toolkits import create_python_agent   
from langchain_experimental.tools.python.tool import PythonREPLTool            
from langchain_ollama  import ChatOllama
#from langchain import RetrievalQA, PromptTemplate, LLMChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from PyPDF2 import PdfReader   # now works, because the env has PyPDF2 installed :contentReference[oaicite:1]{index=1}
from langchain.agents import AgentType


In [8]:
model_name = "gemma3:12b"

## Build an agent which has access to a Tool which writes and executes python code

In [2]:
from IPython.display import Markdown

In [ ]:

# Instantiate your LLM
llm = ChatOllama(model = model_name, temperature=0)

# Create the REPL tool
python_repl = PythonREPLTool()

# Spin up a “python agent”
agent = create_python_agent(
    llm=llm,
    tool=python_repl,
    verbose=True,
)

# Ask it to both write & run code
result = agent.run(
    "Compute the vector norm of [3, 4, 5] and return the result.")
print("Result:", result)




> Entering new AgentExecutor chain...
I need to calculate the Euclidean norm (or vector norm) of the given vector [3, 4, 5].
Action: Python_REPL
Action Input: import math
vector = [3, 4, 5]
norm = math.sqrt(sum([x**2 for x in vector]))
print(norm)
Observation: 7.0710678118654755

Thought:I now know the final answer
Final Answer: The vector norm of [3, 4, 5] is approximately 7.071.

> Finished chain.
Result: The vector norm of [3, 4, 5] is approximately 7.071.


In [4]:
create_python_agent?

Signature:
create_python_agent(
    llm: langchain_core.language_models.base.BaseLanguageModel,
    tool: langchain_experimental.tools.python.tool.PythonREPLTool,
    agent_type: langchain.agents.agent_types.AgentType = <AgentType.ZERO_SHOT_REACT_DESCRIPTION: 'zero-shot-react-description'>,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    verbose: bool = False,
    prefix: str = 'You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n',
    agent_executor_kwargs: Optional[Dict[str, Any]] = None,
    **kwargs: Dict[st

#### More advanced functions

In [5]:
prefix: str = 'You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n.'
second_part = """ Here are the sources you are allowed to query: all the files in base folder: '/home/nitish/Documents/github/PublicReportResearch/docs'. 
Decide which file to read by writing code to read all the file names available in the location and then reading the corresponding file. The filenames contain the company name, year and quarter to pull the earnings of the banking firms from. To read pdf files, use PyPDF2. 
Files are named as jpmc_result_2025_q1.pdf, same for citi(citi_result_2025_q1.pdf), boa_result_2025_q1.pdf (for bank of america).

Here is the set of thoughts you should have: 
1. Read the file names in the base folder.
2. Decide which file or files to read based on the company name, year and quarter.
3. Only read the files which will help you answer the question.
4. If you cannot find the file, return "I don't know" as the answer.
5. Read the file and extract the required information.

Repeat till you have the answer to the question. Answer in the tone asked by the user. If the user asks for a detailed answer, be more verbose. If the user asks for a factual or short answer, be more matter of fact

"""

In [15]:

# Instantiate your LLM
llm = ChatOllama(model = model_name, temperature=0.1)

# Create the REPL tool
python_repl = PythonREPLTool()

# Spin up a “python agent”
agent = create_python_agent(
    llm=llm,
    tool=python_repl,
    verbose=True,
    prefix = prefix + second_part, agent_executor_kwargs = dict(handle_parsing_errors=True),
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

# Ask it to both write & run code
'''result = agent.run(
    "How much did citi bank report as revenue in 2024 Q2?")
print("Result:", result)'''


'result = agent.run(\n    "How much did citi bank report as revenue in 2024 Q2?")\nprint("Result:", result)'

In [16]:
result = agent.run(
    "Compare CITI bank's 2025 Q1 earnings report with JPmorgans 2025 Q1 earnings report on top financial metrics in a table.")
print("Result:", result)



> Entering new AgentExecutor chain...
Okay, I need to compare CITI and JPMorgan's 2025 Q1 earnings reports. I'm going to read the relevant PDF files and extract the key financial metrics, then present them in a table.

Action: Python_REPL
Action Input: `import os
import PyPDF2

base_folder = '/home/nitish/Documents/github/PublicReportResearch/docs'
files = [f for f in os.listdir(base_folder) if f.endswith('.pdf')]
print(files)`
Observation: ['citi_result_2024_q3.pdf', 'boa_result_2024_q2.pdf', 'boa_result_2024_q1.pdf', 'boa_result_2024_q4.pdf', 'jpmc_result_2024_q4.pdf', 'jpmc_result_2025_q1.pdf', 'boa_ppt_2024_q2.pdf', 'boa_ppt_2024_q4.pdf', 'boa_ppt_2024_q3.pdf', 'boa_result_2024_q3.pdf', 'citi_result_2024_q2.pdf', 'jpmc_result_2024_q2.pdf', 'citi_result_2025_q1.pdf', 'citi_result_2024_q1.pdf', 'boa_ppt_2025_q1.pdf', 'jpmc_result_2024_q1.pdf', 'boa_result_2025_q1.pdf', 'citi_result_2024_q4.pdf', 'jpmc_result_2024_q3.pdf']

Thought:Okay, I have the list of files. I need to read `cit

KeyboardInterrupt: 

In [19]:
from langchain_ollama import ChatOllama
from langchain.output_parsers.structured import StructuredOutputParser, ResponseSchema
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import initialize_agent, Tool, AgentType

# 1️⃣ Define the JSON schema for your agent’s tool calls
response_schemas = [
    ResponseSchema(name="action",       description="The exact name of the tool to call, e.g. 'python_repl'"),
    ResponseSchema(name="action_input", description="A JSON object of inputs matching that tool’s signature"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# 2️⃣ Instantiate the Ollama chat model with structured‐output enforcement
llm = ChatOllama(model="qwen2.5:7b", temperature=0).with_structured_output(output_parser)

# 3️⃣ Prepare the Python REPL tool
python_repl = PythonREPLTool()

# 4️⃣ Wrap it as a LangChain Tool, embedding the format instructions in its description
tools = [
    Tool.from_function(
        python_repl.run,
        name="python_repl",
        description=(
            "Executes Python code in a REPL and returns stdout. "
            "Your response MUST follow this JSON schema:\n" + format_instructions
        )
    )
]

# 5️⃣ Initialize a ReAct‐style agent with strict structured‐output enforcement
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    output_parser=output_parser,
    verbose=True,
)

# 6️⃣ Run your task (note: prompt passed as keyword `input`)
result = agent.run(
    input=(
        "Write a function `fib(n)` that returns the nth Fibonacci number, "
        "then compute `fib(10)` and return ONLY the numeric result."
    )
)
print("Result:", result)




> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'\n\t"action"'}

In [13]:
PythonREPLTool?

Init signature:
PythonREPLTool(
    *,
    name: str = 'Python_REPL',
    description: str = 'A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.',
    args_schema: Annotated[Union[type[pydantic.main.BaseModel], dict[str, Any], NoneType], SkipValidation()] = None,
    return_direct: bool = False,
    verbose: bool = False,
    callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    tags: Optional[list[str]] = None,
    metadata: Optional[dict[str, Any]] = None,
    handle_tool_error: Union[bool, str, Callable[[langchain_core.tools.base.ToolException], str], NoneType] = False,
    handle_validation_error: Union[bool, str, Callable[[Union[pydantic_core._pydantic_core.ValidationError, py

In [21]:
Markdown(result)

I don't know.

### Combine this with RAG

In [ ]:


# 1️⃣ Your LLM
llm = OpenAI(temperature=0)

# 2️⃣ A vectorstore + retriever for RAG
emb = OpenAIEmbeddings()
vectordb = Chroma.from_existing_collection("my_collection", embedding_function=emb)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

# 3️⃣ Wrap RAG as a Tool
rag_tool = Tool.from_function(
    rag_chain.run,
    name="rag_retriever",
    description="Use this to fetch relevant context from your document store."
)

# 4️⃣ Python REPL tool
python_repl = PythonREPLTool()

# 5️⃣ Put them both into an agent
tools = [rag_tool, Tool.from_function(python_repl.run, name="python_repl")]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 6️⃣ Run a mixed task
prompt = """
Load the relevant data for 'quarterly earnings analysis' using rag_retriever.
Then write Python code that computes the 90-day moving average of the 'revenue' field
and return the result.
"""
print(agent.run(prompt))


Signature:
create_python_agent(
    llm: langchain_core.language_models.base.BaseLanguageModel,
    tool: langchain_experimental.tools.python.tool.PythonREPLTool,
    agent_type: langchain.agents.agent_types.AgentType = <AgentType.ZERO_SHOT_REACT_DESCRIPTION: 'zero-shot-react-description'>,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    verbose: bool = False,
    prefix: str = 'You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n',
    agent_executor_kwargs: Optional[Dict[str, Any]] = None,
    **kwargs: Dict[st